In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [5]:
df = pd.read_csv('titanic.csv')

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,NaN,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,NaN,3,"Heikkinen, Miss. Laina",female,NaN,0,0,STON/O2. 3101282,NaN,NaN,S
3,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
4,4,NaN,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


# Let's Plan

In [7]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [8]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [9]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,female,16.0,0,1,57.9792,C
734,2,male,NaN,0,0,0.0000,S
382,1,female,42.0,0,0,227.5250,C
705,3,male,25.0,0,0,7.7417,Q
814,2,male,35.0,0,0,10.5000,S


In [10]:
y_train.sample(5)

386    0.0
41     1.0
534    0.0
287    0.0
473    0.0
Name: Survived, dtype: float64

In [11]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')


In [12]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [13]:

# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])


In [14]:
#Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [15]:
# train the model
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [16]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# Pipeline Vs make_pipeline

Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [17]:
# # Alternate Syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [18]:
# train
pipe.fit(X_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Explore the Pipeline

In [ ]:
# Code here
pipe.named_steps

In [19]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [20]:
# Predict
y_pred = pipe.predict(X_test)

NotFittedError: This SelectKBest instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [21]:
y_pred

NameError: name 'y_pred' is not defined

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

# Cross Validation using Pipeline

In [23]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# GridSearch using Pipeline

In [24]:
# # gridsearchcv
# params = {
#     'trf5__max_depth':[1,2,3,4,5,None]
# }

In [25]:
# from sklearn.model_selection import GridSearchCV
# grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
# grid.fit(X_train, y_train)

In [26]:
#grid.best_score_

In [27]:
#grid.best_params_

# Exporting the Pipeline

In [28]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))